In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install gensim


In [3]:
from gensim.models import Word2Vec

In [4]:
df=pd.read_csv('tweet_sentiment.csv')

In [5]:
df.head()

,tweet,sentiment
0,The event starts at 5 PM.,neutral
1,I hate how this turned out.,negative
2,Fantastic experience!,positive
3,Fantastic experience!,positive
4,This is the worst thing ever!,negative


In [6]:
df.tail()

,tweet,sentiment
995,It’s cloudy outside.,neutral
996,Great job by the team!,positive
997,I'm so happy about the news!,positive
998,It ruined my whole day.,negative
999,I'm sick of this happening.,negative


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      1000 non-null   object
 1   sentiment  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [8]:
df.describe()

,tweet,sentiment
count,1000,1000
unique,17,3
top,Fantastic experience!,positive
freq,71,396


In [9]:
import spacy

In [10]:
nlp=spacy.load('en_core_web_sm')

In [11]:
doc=df['tweet'].apply(lambda x: nlp(x))

In [12]:
def preprocess(texts):
  tokens=[token.text for token in texts if not token.is_stop and not token.is_punct]
  return tokens

In [13]:
final=doc.apply(lambda x: preprocess(x))

In [14]:


final_text=final.apply(lambda x: ' '.join(x))

In [15]:
model = Word2Vec(final_text, vector_size=100, window=5, min_count=1, workers=4)


In [16]:
max_len = 100
X = []

for sentence in final_text:
    sentence_vecs = []
    for word in sentence:
        if word in model.wv:
            sentence_vecs.append(model.wv[word])
    # Pad to fixed length
    while len(sentence_vecs) < max_len:
        sentence_vecs.append(np.zeros(model.vector_size))
    sentence_vecs = sentence_vecs[:max_len]  # truncate if longer
    X.append(sentence_vecs)

X = np.array(X)  # Shape: (num_sentences, 100, 100)


In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
le=LabelEncoder()

In [19]:
y=le.fit_transform(df['sentiment'])

In [20]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding, Dropout

In [21]:
model=Sequential([

    LSTM(65,return_sequences=True),
    Dropout(0.2),
    LSTM(2,),
    Dense(1,activation='sigmoid')
])


In [22]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [25]:
model.fit(x_train,y_train,epochs=25)

Epoch 1/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.2969 - loss: 0.5872
Epoch 2/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - accuracy: 0.3371 - loss: 0.1426
Epoch 3/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.3207 - loss: 0.1073
Epoch 4/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.2765 - loss: 0.0844
Epoch 5/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 183ms/step - accuracy: 0.2984 - loss: 0.0456
Epoch 6/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 173ms/step - accuracy: 0.3115 - loss: 0.0310
Epoch 7/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 133ms/step - accuracy: 0.3336 - loss: 0.0412
Epoch 8/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 128ms/step - accuracy: 0.2920 - loss: -0.0346
Epoch 9/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.2958 - loss: -0.0573
Epoch 10/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.3141 - loss: 0.0340
Epoch 11/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.3241 - loss: -0.0510
Epoch 12/25
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - 

In [27]:
y_pred = model.predict(x_test)
print("Predicted Output:", y_pred)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step
Predicted Output: [[0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.9306032]
 [0.930